# Optimal Portfolio Creation

## Introduction

In this notebook, I will discuss the process of creating optimal risky portfolio from the securities in Dhaka Stock Exchange. This is a course requirements for this semesters Investment and Portfolio Analysis of IBA MBA. The course is taken by Mrs Shakila Yasmin. I will discuss the process from collecting historical data, storing in usable format and preprocessing for analysis to creating investable portfolio. Its will be a wild ride. Hope you all enjoy it.  

## Outlines

- Scrap or collect historical data of all securities in DSE
- Store it MongoDb for easy collection and use
- Create an nodejs api for easy collection to use in Pandas [optional]
- Get Data for specific securities, do a base line analysis and derive decision particulars
- From particular of all the securities sort the best numbers of securities
- Get/Scrap fundamentals data from annual report and store in database and create retrieve api
- Do fundamental analysis of securities to find undervalued and overvalued stocks
    - Divident Discount Model
    - Free Cashflow Model
    - Earnings Multiplier Model
    - Ratio Analysis
- Do technical analysis with prophet, regression and machine learning techniques
- Select the best 5 and make all optimum portfolio combination of two stocks
- Select the best one and allocate funds along with risk free asset based on investor risk aversion
    - Figure out riskfree rate
    - Figure out risk aversion ( score card method )
- Invest and update the process along the way


__Special Requirement__
- Use latex for Report and use versioning
- Make a visualization frontend with vue

## Lets Begin

### Scraping the Data

In [17]:
# importing modules necessary
import pandas as pd
import requests as req
from pymongo import MongoClient
from bs4 import BeautifulSoup

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
db = client.dsedb
daily_stock = db.daily_stock

In [5]:
daily_stock.insert_one({'symbol': "XBTUSD"})

In [6]:
help(req.post)

Help on function post in module requests.api:

post(url, data=None, json=None, **kwargs)
    Sends a POST request.
    
    :param url: URL for the new :class:`Request` object.
    :param data: (optional) Dictionary, list of tuples, bytes, or file-like
        object to send in the body of the :class:`Request`.
    :param json: (optional) json data to send in the body of the :class:`Request`.
    :param \*\*kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response



In [7]:
url = "https://www.dsebd.org/day_end_archive.php"
start = "2019-09-01"
end = "2019-09-01"
symbol = "All Instrument"
button = "View Day End Archive"
form = {
    "DayEndSumDate1" : start,
    "DayEndSumDate2" : end,
    "Symbol" : symbol,
    "ViewDayEndArchive" : button
}

In [18]:
html = req.post(url, data = form)
source = BeautifulSoup(html.content)

In [28]:
table = source.select_one("body > table:nth-child(9) > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > table ")

body > table:nth-child(9) > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > table
/html/body/table[2]/tbody/tr/td[2]/table/tbody/tr[2]/td[1]/table

In [57]:
df = pd.read_html(table.prettify())[0]

In [58]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,#,DATE,TRADING CODE,LTP*,HIGH,LOW,OPENP*,CLOSEP*,YCP,TRADE,VALUE (mn),VOLUME
1,1,2019-09-01,1JANATAMF,4.1,4.2,4.1,4.2,4.1,4.2,98,1.471,357297
2,2,2019-09-01,1STPRIMFMF,15.7,16.5,15.1,16.5,15.3,16.2,326,8.39,534662
3,3,2019-09-01,AAMRANET,49.8,51.8,49.8,50.1,49.9,50,98,2.711,54004
4,4,2019-09-01,AAMRATECH,25,25.8,24.9,24.9,25,25.3,55,0.626,25036


In [63]:
help(df.rename)

Help on method rename in module pandas.core.frame:

rename(mapper=None, index=None, columns=None, axis=None, copy=True, inplace=False, level=None, errors='ignore') method of pandas.core.frame.DataFrame instance
    Alter axes labels.
    
    Function / dict values must be unique (1-to-1). Labels not contained in
    a dict / Series will be left as-is. Extra labels listed don't throw an
    error.
    
    See the :ref:`user guide <basics.rename>` for more.
    
    Parameters
    ----------
    mapper : dict-like or function
        Dict-like or functions transformations to apply to
        that axis' values. Use either ``mapper`` and ``axis`` to
        specify the axis to target with ``mapper``, or ``index`` and
        ``columns``.
    index : dict-like or function
        Alternative to specifying axis (``mapper, axis=0``
        is equivalent to ``index=mapper``).
    columns : dict-like or function
        Alternative to specifying axis (``mapper, axis=1``
        is equivalent 

In [66]:
df = df.rename(df.iloc[0], axis=1)[1:]

In [68]:
df = df.drop('#', axis=1)

In [72]:
df['Date'] = pd.to_datetime(df['DATE'])

In [76]:
df = df.set_index('Date')

In [75]:
df.head()

,DATE,TRADING CODE,LTP*,HIGH,LOW,OPENP*,CLOSEP*,YCP,TRADE,VALUE (mn),VOLUME,Date
1,2019-09-01,1JANATAMF,4.1,4.2,4.1,4.2,4.1,4.2,98,1.471,357297,2019-09-01
2,2019-09-01,1STPRIMFMF,15.7,16.5,15.1,16.5,15.3,16.2,326,8.39,534662,2019-09-01
3,2019-09-01,AAMRANET,49.8,51.8,49.8,50.1,49.9,50,98,2.711,54004,2019-09-01
4,2019-09-01,AAMRATECH,25,25.8,24.9,24.9,25,25.3,55,0.626,25036,2019-09-01
5,2019-09-01,ABB1STMF,4.1,4.2,4.1,4.2,4.1,4.1,49,1.963,478736,2019-09-01


In [79]:
df = df.drop('DATE', axis=1)

In [80]:
df.head()

,TRADING CODE,LTP*,HIGH,LOW,OPENP*,CLOSEP*,YCP,TRADE,VALUE (mn),VOLUME
Date,,,,,,,,,,
2019-09-01,1JANATAMF,4.1,4.2,4.1,4.2,4.1,4.2,98,1.471,357297
2019-09-01,1STPRIMFMF,15.7,16.5,15.1,16.5,15.3,16.2,326,8.39,534662
2019-09-01,AAMRANET,49.8,51.8,49.8,50.1,49.9,50,98,2.711,54004
2019-09-01,AAMRATECH,25,25.8,24.9,24.9,25,25.3,55,0.626,25036
2019-09-01,ABB1STMF,4.1,4.2,4.1,4.2,4.1,4.1,49,1.963,478736


In [84]:
df = df.rename(columns = {'Date': 'DATE'})

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 586 entries, 2019-09-01 to 2019-09-01
Data columns (total 10 columns):
TRADING CODE    586 non-null object
LTP*            586 non-null object
HIGH            586 non-null object
LOW             586 non-null object
OPENP*          586 non-null object
CLOSEP*         586 non-null object
YCP             586 non-null object
TRADE           586 non-null object
VALUE (mn)      586 non-null object
VOLUME          586 non-null object
dtypes: object(10)
memory usage: 50.4+ KB
